<a href="https://colab.research.google.com/github/amaye15/CanineNet/blob/main/code/model_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
%pip install huggingface_hub torchview mlflow datasets==2.15.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


# HuggingFace Login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Set Up

In [ ]:
import io
import os
import cv2
import datasets
import torch
import torchview
import pyarrow_hotfix
import mlflow
import mlflow.pytorch
import graphviz
import uuid

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.optim as optim

from PIL import Image
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from datasets import load_dataset, ClassLabel, Value
from functools import partial
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torchview import draw_graph


pyarrow_hotfix.uninstall()
graphviz.set_jupyter_format('png')

os.environ['MLFLOW_TRACKING_USERNAME'] = "andrewmayes14"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "ccb096afadd26486a787461f3495219662998c4b"
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = "mlflow"

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME']
                         + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

mlflow.set_experiment("CanineNetV2")

# Constants
DATASET = "Alanox/stanford-dogs"
REMOVE_COLS_V1 = ["name", "annotations"]
REMOVE_COLS_V2 = ["name", "annotations", "image"]
NAME_COLS = {"index": "label", 0: "count"}
SPLIT = 0.2

# Load the dataset
dataset = datasets.load_dataset(DATASET, split="full").remove_columns(REMOVE_COLS_V1)# .train_test_split(test_size=SPLIT, stratify_by_column="target") #, streaming=True)

### This is a bit of a work aroound to get even class seperation
target_df = datasets.load_dataset(DATASET, split="full").remove_columns(REMOVE_COLS_V2).to_pandas().value_counts().reset_index().reset_index().rename(columns=NAME_COLS)
# Convert the target column to a list and find unique classes
unique_classes = set(target_df['target'])
# Define the ClassLabel feature
class_label_feature = ClassLabel(num_classes=len(unique_classes), names=list(unique_classes), id = list(range(len(unique_classes))))
# Update the dataset schema to use ClassLabel for the target column
dataset = dataset.cast_column('target', class_label_feature)
###

# Split Dataset
dataset = dataset.train_test_split(test_size=SPLIT, stratify_by_column = "target")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating full split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/20580 [00:00<?, ? examples/s]

# Model

In [ ]:
# Load the model and the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224',
                                                  num_labels = len(target_df["label"].tolist()),
                                                  id2label = dict(zip(dataset["train"].features["target"].id, dataset["train"].features["target"].names)),
                                                  label2id = dict(zip(dataset["train"].features["target"].names, dataset["train"].features["target"].id)),
                                                  ignore_mismatched_sizes=True)



preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([120]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([120, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def transforms(examples, transformation: bool = False, ops: int = 10):
  if transformation:
    transformation = v2.Compose([v2.RandAugment(num_ops = ops)])
    examples["image"] = [feature_extractor(transformation(image), return_tensors = "pt")["pixel_values"].squeeze() for image in examples["image"]]
  else:
    examples["image"] = [feature_extractor(image, return_tensors = "pt")["pixel_values"].squeeze() for image in examples["image"]]
  return examples

# Fine Tune - No Transformation

In [ ]:
# Start run
run = mlflow.start_run()

# Train Config
EPOCHS = 10
LEARNING_RATE = 5e-5
BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRANSFORM = False

# Criteria & Optimiser
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = EPOCHS * dataset["train"].num_rows
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_training_steps * 0.01, num_training_steps=num_training_steps)

# Device
model.to(DEVICE)

# Log parameters
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("learning_rate", LEARNING_RATE)
mlflow.log_param("batch_size", BATCH_SIZE)
mlflow.log_param("image_transformation", TRANSFORM)
# mlflow.log_param("layers", len(convLayers))

train_step = 0
test_step = 0

if TRANSFORM:
  # Transformation and dataset settings as before
  dataset["train"].set_transform(partial(transforms, transformation = True,))
  dataset["test"].set_transform(transforms)
else:
  # Transformation and dataset settings as before
  dataset.set_transform(transforms)

# Model, Criterion, Optimizer setup remains the same
for epoch in range(EPOCHS):

  first_epoch = (epoch == 0)
  last_epoch = (epoch == (EPOCHS - 1))

  # Train
  print("Train")
  model.train()
  for batch in DataLoader(dataset["train"], batch_size=BATCH_SIZE, shuffle=True):
      optimizer.zero_grad()
      output = model(batch["image"].float().to(DEVICE))
      loss = criterion(output.logits, batch["target"].to(DEVICE))
      loss.backward()
      optimizer.step()
      scheduler.step()

      train_step += 1

      # Compute training metrics
      train_accuracy = accuracy_score(batch["target"].numpy(), output.logits.argmax(dim=1).cpu().numpy())
      train_precision, train_recall, train_f1, train_support = precision_recall_fscore_support(batch["target"].numpy(),
                                                                                               output.logits.argmax(dim=1).cpu().numpy(),
                                                                                               average='macro',
                                                                                               zero_division = 0.0)
      # Log training metrics
      mlflow.log_metrics({'train_loss': loss.item(),
                          'train_accuracy': train_accuracy,
                          'train_precision': train_precision,
                          'train_recall': train_recall,
                          'train_f1': train_f1,}, step = train_step)

      print({'train_loss': loss.item(),
              'train_accuracy': train_accuracy,
              'train_precision': train_precision,
              'train_recall': train_recall,
              'train_f1': train_f1,})

  # Test
  print("Test")
  model.eval()

  if last_epoch:
    all_test_labels = []
    all_test_preds = []

  with torch.no_grad():
      for batch in DataLoader(dataset["test"], batch_size=BATCH_SIZE, shuffle=False):
          output = model(batch["image"].squeeze(dim=0).float().to(DEVICE))
          loss = criterion(output.logits, batch["target"].to(DEVICE))

          if last_epoch:
            all_test_labels.extend(batch["target"].numpy().tolist())
            all_test_preds.extend(output.logits.argmax(dim=1).cpu().numpy().tolist())

          test_step += 1

          # Compute testing metrics
          test_accuracy = accuracy_score(batch["target"].numpy(), output.logits.argmax(dim=1).cpu().numpy())
          test_precision, test_recall, test_f1, test_support = precision_recall_fscore_support(batch["target"].numpy(),
                                                                                               output.logits.argmax(dim=1).cpu().numpy(),
                                                                                               average='macro',
                                                                                               zero_division = 0.0)
          # Log testing metrics
          mlflow.log_metrics({'test_loss': loss.item(),
                              'test_accuracy': test_accuracy,
                              'test_precision': test_precision,
                              'test_recall': test_recall,
                              'test_f1': test_f1,}, step = test_step)

          print({'test_loss': loss.item(),
                              'test_accuracy': test_accuracy,
                              'test_precision': test_precision,
                              'test_recall': test_recall,
                              'test_f1': test_f1,})
  if last_epoch:
    id2labels = dict(zip(dataset["train"].features["target"].id, dataset["train"].features["target"].names))
    sorted_ids = sorted(id2labels.keys())
    class_names = [id2labels[i] for i in sorted_ids]

    # Log Confusion Matrix as an artifact
    cm = confusion_matrix(all_test_labels, all_test_preds)
    plt.figure(figsize=(30,21))
    sns.heatmap(cm, annot=False, xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(f'Confusion Matrix at Epoch {epoch}')
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")

# End run
mlflow.end_run()
feature_extractor.push_to_hub("ViT-Standford-Dogs")
model.push_to_hub("ViT-Standford-Dogs")


# Fine Tune - Transformation

In [ ]:
# Start run
run = mlflow.start_run()

# Train Config
EPOCHS = 10
LEARNING_RATE = 5e-5
BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRANSFORM = True

# Criteria & Optimiser
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = EPOCHS * dataset["train"].num_rows
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_training_steps * 0.01, num_training_steps=num_training_steps)

# Device
model.to(DEVICE)

# Log parameters
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("learning_rate", LEARNING_RATE)
mlflow.log_param("batch_size", BATCH_SIZE)
mlflow.log_param("image_transformation", TRANSFORM)
# mlflow.log_param("layers", len(convLayers))

train_step = 0
test_step = 0

if TRANSFORM:
  # Transformation and dataset settings as before
  dataset["train"].set_transform(partial(transforms, transformation = True,))
  dataset["test"].set_transform(transforms)
else:
  # Transformation and dataset settings as before
  dataset.set_transform(transforms)

# Model, Criterion, Optimizer setup remains the same
for epoch in range(EPOCHS):

  if TRANSFORM:
    # Transformation and dataset settings as before
    dataset["train"].set_transform(partial(transforms, transformation = True,))

  first_epoch = (epoch == 0)
  last_epoch = (epoch == (EPOCHS - 1))

  # Train
  print("Train")
  model.train()
  for batch in DataLoader(dataset["train"], batch_size=BATCH_SIZE, shuffle=True):
      optimizer.zero_grad()
      output = model(batch["image"].float().to(DEVICE))
      loss = criterion(output.logits, batch["target"].to(DEVICE))
      loss.backward()
      optimizer.step()
      scheduler.step()

      train_step += 1

      # Compute training metrics
      train_accuracy = accuracy_score(batch["target"].numpy(), output.logits.argmax(dim=1).cpu().numpy())
      train_precision, train_recall, train_f1, train_support = precision_recall_fscore_support(batch["target"].numpy(),
                                                                                               output.logits.argmax(dim=1).cpu().numpy(),
                                                                                               average='macro',
                                                                                               zero_division = 0.0)
      # Log training metrics
      mlflow.log_metrics({'train_loss': loss.item(),
                          'train_accuracy': train_accuracy,
                          'train_precision': train_precision,
                          'train_recall': train_recall,
                          'train_f1': train_f1,}, step = train_step)

      print({'train_loss': loss.item(),
              'train_accuracy': train_accuracy,
              'train_precision': train_precision,
              'train_recall': train_recall,
              'train_f1': train_f1,})

  # Test
  print("Test")
  model.eval()

  if last_epoch:
    all_test_labels = []
    all_test_preds = []

  with torch.no_grad():
      for batch in DataLoader(dataset["test"], batch_size=BATCH_SIZE, shuffle=False):
          output = model(batch["image"].squeeze(dim=0).float().to(DEVICE))
          loss = criterion(output.logits, batch["target"].to(DEVICE))

          if last_epoch:
            all_test_labels.extend(batch["target"].numpy().tolist())
            all_test_preds.extend(output.logits.argmax(dim=1).cpu().numpy().tolist())

          test_step += 1

          # Compute testing metrics
          test_accuracy = accuracy_score(batch["target"].numpy(), output.logits.argmax(dim=1).cpu().numpy())
          test_precision, test_recall, test_f1, test_support = precision_recall_fscore_support(batch["target"].numpy(),
                                                                                               output.logits.argmax(dim=1).cpu().numpy(),
                                                                                               average='macro',
                                                                                               zero_division = 0.0)
          # Log testing metrics
          mlflow.log_metrics({'test_loss': loss.item(),
                              'test_accuracy': test_accuracy,
                              'test_precision': test_precision,
                              'test_recall': test_recall,
                              'test_f1': test_f1,}, step = test_step)

          print({'test_loss': loss.item(),
                              'test_accuracy': test_accuracy,
                              'test_precision': test_precision,
                              'test_recall': test_recall,
                              'test_f1': test_f1,})
  if last_epoch:
    id2labels = dict(zip(dataset["train"].features["target"].id, dataset["train"].features["target"].names))
    sorted_ids = sorted(id2labels.keys())
    class_names = [id2labels[i] for i in sorted_ids]

    # Log Confusion Matrix as an artifact
    cm = confusion_matrix(all_test_labels, all_test_preds)
    plt.figure(figsize=(30,21))
    sns.heatmap(cm, annot=False, xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(f'Confusion Matrix at Epoch {epoch}')
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")

# End run
mlflow.end_run()
feature_extractor.push_to_hub("ViT-Standford-Dogs")
model.push_to_hub("ViT-Standford-Dogs")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train
{'train_loss': 4.952563285827637, 'train_accuracy': 0.0, 'train_precision': 0.0, 'train_recall': 0.0, 'train_f1': 0.0}
{'train_loss': 5.028340816497803, 'train_accuracy': 0.015625, 'train_precision': 0.00684931506849315, 'train_recall': 0.00684931506849315, 'train_f1': 0.00684931506849315}
{'train_loss': 5.088619709014893, 'train_accuracy': 0.0, 'train_precision': 0.0, 'train_recall': 0.0, 'train_f1': 0.0}
{'train_loss': 4.996757507324219, 'train_accuracy': 0.0, 'train_precision': 0.0, 'train_recall': 0.0, 'train_f1': 0.0}
{'train_loss': 4.8988823890686035, 'train_accuracy': 0.015625, 'train_precision': 0.013513513513513514, 'train_recall': 0.013513513513513514, 'train_f1': 0.013513513513513514}
{'train_loss': 4.9119720458984375, 'train_accuracy': 0.015625, 'train_precision': 0.006756756756756757, 'train_recall': 0.013513513513513514, 'train_f1': 0.009009009009009009}
{'train_loss': 4.984621524810791, 'train_accuracy': 0.0, 'train_precision': 0.0, 'train_recall': 0.0, 'train_f1':